# Урок 6. Введение в библиотеки глубокого обучения: TensorFlow, Keras

На предыдущих уроках мы познакомились с базовыми понятиями того, что из себя представляют нейронные сети. Мы самостоятельно реализовали однослойные (shallow) и многослойные (deep) полносвязные нейронные сети (fully connected NN). Однако на практике построение нейронных сетей с нуля не производят. На сегодняшний день доступно множество библиотек на разных языках программирования, которые позволяют эффективно решать задачи по deep learning. Среди них:
 - Tensorflow (Google);
 - Theano (University of Montreal);
 - PyTorch (Facebook);
 - Caffe (UC Berkeley);
 - MXNet (Apache);
 - CNTK (Microsoft).
 
Все эти библиотеки имеют поддержку CUDA, то есть можно проводить вычисления на видеокартах nvidia. Интерфейс этих библиотек может достаточно сильно отличаться друг от друга по синтаксису, однако суть везде та же самая: библиотеки реализовывают шаг forward и backward propagation и осуществляют процесс обучения с заданной функцией ошибки и заданными гиперпараметрами (в т.ч. оптимизатором).

## Keras

$\textbf{Keras}$ тоже часто называют библиотекой глубокого обучения, однако это неверно. Keras - это библиотека и интерфейс, упрощающий взаимодействие между пользователем и непосредственно библиотекой глубокого обучения (Keras поддерживает TensorFlow, Theano и CNTK). Его интерфейс максимально простой и интуитивно понятный, вследствие чего он хорошо подходит для быстрого проектирования нейронной сети.

## Пример задач на TensorFlow

Ниже приведена реализация функции ошибки MSE

$$loss = \mathcal{L}(\hat{y}, y) = (\hat y^{(i)} - y^{(i)})^2: \tag{1}$$

```python
y_hat = tf.constant(36, name='y_hat')            # Define y_hat constant. Set to 36.
y = tf.constant(39, name='y')                    # Define y. Set to 39

loss = tf.Variable((y - y_hat)**2, name='loss')  # Create a variable for the loss

init = tf.global_variables_initializer()         # When init is run later (session.run(init)),
                                                 # the loss variable will be initialized and ready to be computed
with tf.Session() as session:                    # Create a session and print the output
    session.run(init)                            # Initializes the variables
    print(session.run(loss))                     # Prints the loss

```

Логика TensorFlow в следующем: сначала мы описываем последовательность вычислений, но при этом никакого подсчета не происходит. Далее мы должны начать т.н. "сессию" (session): во время сессии будут последовательно выполняться описанные нами ранее вычисления.

### Пример 1. Минимизация функции $y = (w-5)^2$ по $w$

In [0]:
import numpy as np
import tensorflow as tf

In [2]:
w = tf.Variable(0,dtype = tf.float32)
cost = tf.add(tf.add(w**2, tf.multiply(-10.,w)), 25)
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
print(session.run(w))

W0723 13:41:59.077844 139668220368768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


0.0


In [3]:
session.run(train)
print(session.run(w))

0.099999994


In [4]:
for i in range(100):
  session.run(train)
  print(session.run(w))

0.198
0.29404
0.3881592
0.480396
0.5707881
0.6593723
0.7461849
0.83126116
0.91463596
0.99634326
1.0764164
1.154888
1.2317903
1.3071545
1.3810115
1.4533913
1.5243235
1.593837
1.6619602
1.728721
1.7941467
1.8582637
1.9210985
1.9826765
2.0430229
2.1021624
2.160119
2.2169166
2.2725782
2.3271267
2.3805842
2.4329727
2.4843132
2.534627
2.5839343
2.6322556
2.6796105
2.7260182
2.7714977
2.8160677
2.8597465
2.9025514
2.9445004
2.9856105
3.0258982
3.0653803
3.1040728
3.1419914
3.1791515
3.2155685
3.2512572
3.286232
3.3205073
3.3540971
3.387015
3.4192748
3.4508893
3.4818716
3.5122342
3.5419896
3.5711498
3.599727
3.6277323
3.6551776
3.682074
3.7084327
3.7342641
3.759579
3.7843874
3.8086996
3.8325257
3.8558753
3.8787577
3.9011827
3.923159
3.9446957
3.9658017
3.9864857
4.006756
4.026621
4.046088
4.0651665
4.0838633
4.102186
4.1201425
4.1377397
4.154985
4.171885
4.1884475
4.2046785
4.220585
4.236173
4.2514496
4.2664204
4.281092
4.29547
4.309561
4.3233695
4.336902
4.350164


**Замечание 1.** На примере видно, что теперь нам можно не заботиться о процедуре взятия производных - это выполняет функция minimize класса GradientDescentOptimizer. На практике это означает, что нам достаточно реализовать шаг forward propagation, а backpropagation будет выполнен автоматически.

**Замечание 2.** Данная функция зависела только от одной переменной, значение которой было заранее известно. Однако функция ошибки в реальных задач глубокого обучения зависит еще и от входных данных, которые заранее неизвестны (на этапе проектирования нейросети). Чтобы обеспечить возможность работы с такими данными (грубо говоря, с входным массивом X), в TensorFlow существует специальный тип данных

```python
tf.placeholder(dtype, shape),
```
где dtype - это тип данных (int, float16, float32), а shape - это размерность входного массива.

### Пример 2. Минимизация функции  $𝑦=x_0\cdot w^2 - x_1 \cdot w + x_2$  по  𝑤

Пусть теперь функция ошибки определяется не только параметром $w$, но и даннымы $x$. Рассмотрим, как реализовать этот пример, используя тип данных tf.placeholder.

In [5]:
coefficients = np.array([[1.,10.,25.]])

w = tf.Variable(0,dtype = tf.float32)
x = tf.placeholder(dtype = tf.float32, shape = [1,3])
cost = x[0][0] * w**2 - x[0][1] * w + x[0][2]
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)


init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
print(session.run(w))

0.0


In [6]:
for i in range(1000):
  session.run(train, feed_dict = {x:coefficients})
print(session.run(w))

4.999988


Написание программ на TensorFlow можно описать несколькими шагами:

1. Объявить переменные Tensors (variables), над которыми никаких операций совершено пока не будет. 
2. Определить математические операции между этими Tensors.
3. ИнициализироватьTensors. 
4. Создать сессию. 
5. Запустить сессию, в процессе которой будут непосредственно выполняться вышеописанные вычисления над переменными Tensors. 

### Пример 3. Что будет, если не запустить Session?

In [7]:
a = tf.constant(2)
b = tf.constant(10)
c = tf.multiply(a,b)
print(c)

Tensor("Mul_3:0", shape=(), dtype=int32)


In [8]:
sess = tf.Session()
print(sess.run(c))

20


### Пример 4.  One-hot Encoding. Многоклассовая классификация и функция softmax

<img src="https://drive.google.com/uc?export=view&id=1J0WflUO8DxWyQuHxcOQYBE1GhfEyOdeE" style="width:665px;height:320px;">

Пусть перед нами поставлена задача многоклассовой классификации. Например, нам надо предсказать цвет светофора по каким-то входным данным. Тогда классов будет три: [красный, желтый, зеленый]. Мы уже не можем воспользоваться единственным числом на выходе нейросети, чтобы получить ответ. Необходимо, чтобы нейросеть выдавала выход размерности 3 (3 в данной задаче). Тогда необходимо каждый цвет светофора тоже закодировать вектором. Простейший способ это сделать - one-hot encoding. Тогда:
 - красный = $(1, 0, 0)^T$;
 - желтый = $(0,1,0)^T$;
 - зеленый = $(0,0,1)^T$.
 
На выходе нейросети будет трехмерный вектор, i-ую координату которого можно интерпретировать как вероятность принадлежности объекта к i-ому классу. 
Используется следующая функция потерь:

$$J(y, \hat y) = -\frac{1}{N} \sum_{s\in S} \sum_{c \in C} 1_{s\in c} \log {p(s \in c)}.$$

Сумма всех координат должна равняться единице:

$$\forall l \in[1,N]  \quad\sum_{i=1}^m y_i^{(l)} = 1,$$

где N -кол-во объектов, l -кол-во классов. Чтобы такое требование выполнялось, вводится новая функция активации, которая называется $\textit{Softmax}:$

$$Softmax(z)_i = \frac{\exp{(z_i)}}{\sum_{j=1}^m \exp{(z_j)}}, \quad i \in [1,m]. $$

Таким образом, требования, чтобы сумма всех координат равнялась единице, автоматически выполняется (равно как и то, что вероятность - это неотрицательная величина).

В TensorFlow есть инструмент, которые позволяет преобразовать данные y в one-hot векторы:

- tf.one_hot(labels, depth, axis).

In [0]:
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    # Create a tf.constant equal to C (depth), name it 'C'.
    C = tf.constant(C, name = 'C')
    
    # Use tf.one_hot, be careful with the axis
    one_hot_matrix = tf.one_hot(labels, C, axis=0)
    
    # Create the session
    sess = tf.Session()
    
    # Run the session
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session
    sess.close()
    
    
    return one_hot

In [10]:
labels = np.array([1,2,3,0,2,1])
one_hot = one_hot_matrix(labels, C = 4)
print ("one_hot = " + str(one_hot))
print(type(one_hot))

one_hot = [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]
<class 'numpy.ndarray'>


### Пример 5. tf.layers

В TensorFlow можно либо прописывать самому цепочку вычислений, происходящих на каждом слое, либо воспользоваться готовой реализацией из раздела tf.layers. Мы пока знакомы только с полносвязными нейросетями (fully connected NN), их слои называются Dense layers. Пример внизу показывает, как работает такой слой (без шага обучения!):

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1, kernel_initializer=tf.zeros_initializer())
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

W0723 08:13:59.179983 140057250158464 deprecation.py:323] From <ipython-input-11-7d9f41f4ec6c>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


[[0.]
 [0.]]


### Пример 6. Линейная регрессия

Линейная регрессия - это задача машинного обучения, заключающаяся в линейной аппроксимации целевой переменной y:

$$ y = Wx + b, \quad J = \sum(\hat y_i - y_i)^2$$

In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(1000):
  _, loss_value = sess.run((train, loss))
  #print(loss_value)

print(sess.run(y_pred))

W0723 08:14:00.157789 140057250158464 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[[-0.04734534]
 [-1.0229421 ]
 [-1.9985389 ]
 [-2.9741354 ]]


### Задание 1. Реализация линейной функции  $y = W \cdot x + b$

Первым заданием будет подсчет следующего выражения: $Y = WX + b$, где $W$и  $X$ - это случайные матрицы, а b - случайный вектор. 

**Задание**: Рассчитать $W\cdot x + b$, где $W, x $, и $b$ получены из стандартного нормального распределения. W имеет размер (4, 3), x - (3,1), b - (4,1). В кач-ве примера показано, как реализовать объявление X с размером (3,1):
```python
X = tf.constant(np.random.randn(3,1), name = "X")

```
Также обратите внимание на следующие функции: 
- tf.matmul(..., ...) - перемножение матриц;
- tf.add(..., ...) - сложение двух объектов;
- np.random.randn(...) - метод numpy для инициализации массивов случайными значениями.

**NB** Обратите внимание на аргумент name функции tf.constant. Не забудьте его тоже правильно указать.

In [0]:
def linear_function():
    """
    Implements a linear function: 
            Initializes W to be a random tensor of shape (4,3)
            Initializes X to be a random tensor of shape (3,1)
            Initializes b to be a random tensor of shape (4,1)
    Returns: 
    result -- runs the session for Y = WX + b 
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (4 lines of code)
    X = tf.constant(np.random.randn(3,1), name='X')
    W = tf.constant(np.random.randn(4,3), name='W')
    b = tf.constant(np.random.randn(4,1), name='b')
    Y = tf.add(tf.matmul(W,X), b)
    ### END CODE HERE ### 
    
    # Create the session using tf.Session() and run it with sess.run(...) on the variable you want to calculate
    
    ### START CODE HERE ###
    sess = tf.Session()
    result = sess.run(Y)
    ### END CODE HERE ### 
    
    # close the session 
    sess.close()

    return result

In [12]:
linear_function()

array([[-2.15657382],
       [ 2.95891446],
       [-1.08926781],
       [-0.84538042]])

### Задание 2. Реализация функции активации (сигмоиды)

Итак, мы реализовали сверху линейную часть шага forward_propagation. Tensorflow уже имеет реализации таких функций активации как сигмоида или ReLU. В данном задании необходимо будет реализовать шаг подсчета сигмоиды. 

Для выполнения этого задания надо будет воспользоваться типом tf.placeholder. При запуске сессии надо будет передать словарь feed_dict со значением z (результат предыдущей функции).
Порядок будет следующим:
 - создать placeholder x;
 - записать вычисления, которые над ним будут проводиться (tf.sigmoid);
 - запустить сессию.

Сессию можно запустить двумя способами: 

**Способ 1:**
```python
sess = tf.Session()
# Вычислительные шаги
result = sess.run(..., feed_dict = {...})
sess.close() # Закрытие сессии
```
** Способ 2:**
```python
with tf.Session() as sess: 
    # Вычислительные шаги
    result = sess.run(..., feed_dict = {...})
    # Нет необходимости явно закрывать сессию :)
```


In [0]:
def sigmoid(z):
    """
    Computes the sigmoid of z
    
    Arguments:
    z -- input value, scalar or vector
    
    Returns: 
    results -- the sigmoid of z
    """
    
    ### START CODE HERE ###
    # Create a placeholder for x. Name it 'x'.
    x = tf.placeholder(dtype = tf.float32, name='x')

    # compute sigmoid(x)
    sigmoid = tf.sigmoid(x)

    # Create a session, and run it.
    # You should use a feed_dict to pass z's value to x. 
    sess = tf.Session()
    
    # Run session and call the output "result"
    result = sess.run(sigmoid, feed_dict={x:z})
    
    sess.close()
    ### END CODE HERE ###
    
    return result

In [20]:
sigmoid([[0],[0]])

array([[0.5],
       [0.5]], dtype=float32)


**Промежуточное summary** 

На данном этапе мы узнали несколько ключевых вещей...
  
1. Мы узнали, что такое tf.Variable и что такое tf.placeholder.
2. Научились объявлять вычислительные операции.
3. Научились создавать сессию.
4. Научились осуществлять сессию с использованием feed_dict - словаря для входных данных. 

### Задание 3 -  Подсчет функции ошибки

Многие функции ошибки также содержатся в библиотеке TensorFlow, в том числе бинарная кросс-энтропия: 
$$ J = - \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log a^{ [2] (i)} + (1-y^{(i)})\log (1-a^{ [2] (i)} )\large ).\small\tag{2}$$


**Задание**: Реализуйте подсчет функции ошибки, используя следующую функцию: 


- `tf.nn.sigmoid_cross_entropy_with_logits(logits = ...,  labels = ...)`

Шаги должны быть следующими: подать сигмоиде входные данные z, посчитать сигмоиду, затем вызвать функцию ошибки, где logits - это вывод нейросети до применения сигмоиды, labels - истинный ответ. Таким образом, будет осуществляться подсчет следующего выражения:

$$- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small\tag{2}$$


In [0]:
def cost(logits, labels):
    """
    Computes the cost using the sigmoid cross entropy
    
    Arguments:
    logits -- vector containing z, output of the last linear unit (before the final sigmoid activation)
    labels -- vector of labels y (1 or 0) 
    
    Note: What we've been calling "z" and "y" in this class are respectively called "logits" and "labels" 
    in the TensorFlow documentation. So logits will feed into z, and labels into y. 
    
    Returns:
    cost -- runs the session of the cost (formula (2))
    """
    
    ### START CODE HERE ### 
    
    # Create the placeholders for "logits" (z) and "labels" (y)
    z = tf.placeholder(dtype=tf.float32, name='z')
    y = tf.placeholder(dtype=tf.float32, name='y')
    
    # Use the loss function 
    cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z,labels=y)
    
    # Create a session (approx. 1 line). See method 1 above.
    sess = tf.Session()
    
    # Run the session (approx. 1 line).
    cost = sess.run(cost, feed_dict={z:logits, y:labels})
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    ### END CODE HERE ###
    
    return cost

In [26]:
cost([-100,-100],[0,0])

array([0., 0.], dtype=float32)

##Keras

Как уже было сказано ранее, Keras - это высокоуровневая оболочка над фреймворком для глубокого обучения (по умолчанию Keras использует ядро TensorFlow). Далее мы рассмотрим несколько примеров проектирования нейросетей на Keras.

### Пример 1. Двуслойная нейросеть для многоклассовой классификации

Используется нейросеть со следующими параметрами:
 - вход имеет размерность (1,100);
 - первый слой имеет 32 нейрона, функция активации ReLU;
 - второй слой имеет 10 нейронов, функция активации Softmax;
 - в качестве оптимизатора используется RMSProp;
 - регуляризаторы не используются;
 - функция ошибки: категориальная кросс-энтропия:
 $$ \large CE = - \large \sum_i^C y_i \log(Softmax(z)_i) = - \large \log(\frac{\exp{(z_i)}}{\sum_{j=1}^m \exp{(z_j)}}),$$
 
 где C -  количество различных классов.

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data

data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Using TensorFlow backend.
W0723 08:14:01.074979 140057250158464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 08:14:01.076247 140057250158464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 08:14:01.079301 140057250158464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 08:14:01.106580 140057250158464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 08:14:01.125079 14005725

Epoch 1/10
1000/1000 [==============================] - 0s 277us/step - loss: 2.3494 - acc: 0.1210
Epoch 2/10
1000/1000 [==============================] - 0s 88us/step - loss: 2.3067 - acc: 0.1340
Epoch 3/10
1000/1000 [==============================] - 0s 91us/step - loss: 2.2935 - acc: 0.1350
Epoch 4/10
1000/1000 [==============================] - 0s 83us/step - loss: 2.2816 - acc: 0.1390
Epoch 5/10
1000/1000 [==============================] - 0s 82us/step - loss: 2.2730 - acc: 0.1600
Epoch 6/10
1000/1000 [==============================] - 0s 82us/step - loss: 2.2636 - acc: 0.1450
Epoch 7/10
1000/1000 [==============================] - 0s 80us/step - loss: 2.2511 - acc: 0.1630
Epoch 8/10
1000/1000 [==============================] - 0s 80us/step - loss: 2.2402 - acc: 0.1740
Epoch 9/10
1000/1000 [==============================] - 0s 79us/step - loss: 2.2312 - acc: 0.1810
Epoch 10/10
1000/1000 [==============================] - 0s 79us/step - loss: 2.2217 - acc: 0.2080


Шаги, проделанные в Keras для обучения нейросети, похожи на те, что мы проделывали в TensorFlow:
 
 - 1) описание вычислительной модели - объект класса Sequential;
 - 2) задание оптимизаторов и функции ошибки - метод compile;
 - 3) обучение модели - метод  fit.
Тем не менее, интерфейс Keras более user-friendly, что наглядно видно на данном примере.

**Важное дополнение** У класса model есть метод summary, который позволяет вывести всю информацию о модели: в каком слое сколько параметров и гиперпараметров, размерность каждого слоя и тд.

In [0]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,562
Trainable params: 3,562
Non-trainable params: 0
_________________________________________________________________
None


В первом слое содержится 32x100 параметров $w_{ij}$ и  32 параметра $b_j$ - всего 3232 обучаемых параметра.

Во втором слое содержится 10х32 параметра $w_{ij}$ и 10 параметров $b_j$ - всего 330 параметров.

Данная архитектура не содержит т.н. non-trainable parameters (параметры, которые зависят от входных данных, но для которых не нужно подбирать оптимальные значения), они бы появились, если бы мы добавили batch_normalization. 

Тогда на каждом слое производился бы подсчет среднего и дисперсии - такие параметры как раз относятся к non-trainable parameters.

###Пример 2. Задача бинарной классификации на Keras

Ниже приведен пример задачи, которую мы уже реализовывали с помощью средств numpy - это задача бинарной классификации (отнесение объекта либо к одному, либо к другому классу).

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

W0723 08:14:02.409227 140057250158464 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 1/20
1000/1000 [==============================] - 0s 253us/step - loss: 0.7075 - acc: 0.5340
Epoch 2/20
1000/1000 [==============================] - 0s 28us/step - loss: 0.7032 - acc: 0.5180
Epoch 3/20
1000/1000 [==============================] - 0s 27us/step - loss: 0.6971 - acc: 0.5160
Epoch 4/20
1000/1000 [==============================] - 0s 30us/step - loss: 0.6981 - acc: 0.5220
Epoch 5/20
1000/1000 [==============================] - 0s 27us/step - loss: 0.6956 - acc: 0.5230
Epoch 6/20
1000/1000 [==============================] - 0s 27us/step - loss: 0.6984 - acc: 0.5000
Epoch 7/20
1000/1000 [==============================] - 0s 31us/step - loss: 0.6958 - acc: 0.5210
Epoch 8/20
1000/1000 [==============================] - 0s 29us/step - loss: 0.6962 - acc: 0.5300
Epoch 9/20
1000/1000 [==============================] - 0s 28us/step - loss: 0.6912 - acc: 0.5280
Epoch 10/20
1000/1000 [==============================] - 0s 30us/step - loss: 0.6940 - acc: 0.5200
Epoch 11/20
1000/1

**ВАЖНО** у модели есть три метода, которые можно спутать друг с другом:
 - model.fit(...) - запуск обучения модели путем минимизации заданной ошибки заданным алгоритмом (на вход подаются x_train и y_train);
 - model.evaluate(...) - проверка качества обученной модели (обучения не происходит) (на вход подаются x_test и y_test);
 - model.predict(...) - получение предсказания модели на новых данных (на вход подается только x)

### Задание на Keras

Реализовать на Keras любую из ранее написанных нами нейросетей (можно даже логистическую регрессию). В качестве входных данных можно взять картинки с коробочками или набор точек из "лепестков".

Добавить в нейросеть регуляризатор dropout.

Воспользоваться оптимизатором adam.

**Обратите внимание!** Если вы добавляете регуляризатор, то надо решить, до или после активации его применять. Чтобы применить регуляризатор до активации, необходимо НЕ передавать аргумент activation при добавлении слоя Dense. Активацию в таком случае надо добавить как слой сразу после dropout-a: keras.layers.Activation(activation=функция_активации)

## PyTorch

PyTorch - еще один фреймворк глубокого обучения. Рассмотрим пример аппроксимации некоторой непрерывной функции с помощью двуслойной нейросети.

In [0]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
    
)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

0 648.992919921875
1 631.898193359375
2 615.2619018554688
3 599.0609741210938
4 583.2977905273438
5 568.0230712890625
6 553.2813110351562
7 538.9461669921875
8 525.0341186523438
9 511.556884765625
10 498.5509948730469
11 485.9294738769531
12 473.6356506347656
13 461.6299743652344
14 449.9319763183594
15 438.528076171875
16 427.52752685546875
17 416.9284973144531
18 406.6570129394531
19 396.62274169921875
20 386.872802734375
21 377.3948974609375
22 368.181396484375
23 359.2437438964844
24 350.5541687011719
25 342.0889587402344
26 333.8456726074219
27 325.7762756347656
28 317.9083557128906
29 310.25799560546875
30 302.7669372558594
31 295.4480895996094
32 288.3083190917969
33 281.3132019042969
34 274.4677429199219
35 267.7706604003906
36 261.2209167480469
37 254.81597900390625
38 248.55584716796875
39 242.4446563720703
40 236.47918701171875
41 230.63636779785156
42 224.92539978027344
43 219.3568115234375
44 213.90985107421875
45 208.5692596435547
46 203.33596801757812
47 198.225463867187

## Summary

На этом уроке мы познакомились с тремя фреймворками для deep-learning:
 - TensorFlow - узнали порядок написания программ, узнали типы данных Variable и placeholder;
 - Keras (который не совсем фреймворк, а скорее интерфейс) - написали в нем простую нейросеть;
 - PyTorch - аналогично написали простую двуслойную нейросеть и посмотрели, как ее обучать.

Помимо этого, мы рассмотрели задачу многоклассовой классификации, а именно:
 - узнали, что такое функция Softmax;
 - узнали, что такое one-hot encoding и как его реализовывать на tensorflow.

In [0]:
!nvidia-smi

Tue Jul 23 08:14:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    30W /  70W |    293MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  